In [52]:
import numpy as np 
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [53]:
df_old = pd.read_csv('CF-Insider-Trading-equities-25-06-2022-to-25-09-2022.csv')
df = pd.read_csv('CF08-07-2022-to-08-10-2022.csv')
print(df_old.columns == df.columns)
df.head()

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True]


,SYMBOL \n,COMPANY \n,REGULATION \n,NAME OF THE ACQUIRER/DISPOSER \n,CATEGORY OF PERSON \n,TYPE OF SECURITY (PRIOR) \n,NO. OF SECURITY (PRIOR) \n,% SHAREHOLDING (PRIOR) \n,TYPE OF SECURITY (ACQUIRED/DISPLOSED) \n,NO. OF SECURITIES (ACQUIRED/DISPLOSED) \n,...,DERIVATIVE TYPE SECURITY \n,DERIVATIVE CONTRACT SPECIFICATION \n,NOTIONAL VALUE(BUY) \n,NUMBER OF UNITS/CONTRACT LOT SIZE (BUY) \n,NOTIONAL VALUE(SELL) \n,NUMBER OF UNITS/CONTRACT LOT SIZE (SELL) \n,EXCHANGE \n,REMARK \n,BROADCASTE DATE AND TIME \n,XBRL \n
0,AUBANK,AU Small Finance Bank Limited,7(2),Ashok Kumar Khandelwal,-,Equity Shares,21794,0.0,NaN,3000.0,...,-,-,0,0,0,0,NaN,-,07-Oct-2022 18:58,https://archives.nseindia.com/corporate/xbrl/I...
1,INFIBEAM,Infibeam Avenues Limited,7(2),Vybe Ventures LLP,Other,Warrants,Nil,0.0,NaN,95000000.0,...,-,-,0,0,0,0,NaN,-,07-Oct-2022 18:16,https://archives.nseindia.com/corporate/xbrl/I...
2,NAUKRI,Info Edge (India) Limited,7(2),Ashish Anand,Employees/Designated Employees,Equity Shares,275,0.0,NaN,100.0,...,-,-,0,0,0,0,NaN,-,07-Oct-2022 17:58,https://archives.nseindia.com/corporate/xbrl/I...
3,NAUKRI,Info Edge (India) Limited,7(2),Ashish Anand,Employees/Designated Employees,Equity Shares,275,0.0,NaN,100.0,...,-,-,0,0,0,0,NaN,-,07-Oct-2022 17:58,https://archives.nseindia.com/corporate/xbrl/I...
4,NAUKRI,Info Edge (India) Limited,7(2),Ashish Anand,Employees/Designated Employees,Equity Shares,275,0.0,NaN,100.0,...,-,-,0,0,0,0,NaN,-,07-Oct-2022 17:58,https://archives.nseindia.com/corporate/xbrl/I...


---
## Renaming All Columns Excluding Unnecessery Ones .

In [54]:
df = df[[
    'SYMBOL \n', 'CATEGORY OF PERSON \n',
    'NO. OF SECURITIES (ACQUIRED/DISPLOSED) \n',
    'VALUE OF SECURITY (ACQUIRED/DISPLOSED) \n', 'MODE OF ACQUISITION \n',
    'ACQUISITION/DISPOSAL TRANSACTION TYPE \n', 'TYPE OF SECURITY (POST) \n',
    '% SHAREHOLDING (PRIOR) \n', '% POST \n'
]]


df.rename(columns={'SYMBOL \n':'SYMBOL','CATEGORY OF PERSON \n':'CATEGORY_OF_PERSON','NO. OF SECURITIES (ACQUIRED/DISPLOSED) \n':'NO_OF_SECURITIES_ACQUIRED','VALUE OF SECURITY (ACQUIRED/DISPLOSED) \n':'VALUE_OF_SECURITY_ACQUIRED','ACQUISITION/DISPOSAL TRANSACTION TYPE \n':'ACQUISITION_TRANSACTION_TYPE','MODE OF ACQUISITION \n':'MODE_OF_ACQUISITION','% SHAREHOLDING (PRIOR) \n':'%_SHAREHOLDING_PRIOR','% POST \n':'%_SHAREHOLDING_POST','TYPE OF SECURITY (POST) \n':'TYPE_OF_SECURITY'},inplace=True)

In [55]:
df.head()

,SYMBOL,CATEGORY_OF_PERSON,NO_OF_SECURITIES_ACQUIRED,VALUE_OF_SECURITY_ACQUIRED,MODE_OF_ACQUISITION,ACQUISITION_TRANSACTION_TYPE,TYPE_OF_SECURITY,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST
0,AUBANK,-,3000.0,1801200,Pledge Creation,Pledge,Equity Shares,0.0,0
1,INFIBEAM,Other,95000000.0,403750000,Preferential Offer,Buy,Warrants,0.0,3.43
2,NAUKRI,Employees/Designated Employees,100.0,354752,Revokation of Pledge,Pledge Revoke,Equity Shares,0.0,0
3,NAUKRI,Employees/Designated Employees,100.0,341198,Pledge Creation,Pledge,Equity Shares,0.0,0
4,NAUKRI,Employees/Designated Employees,100.0,353097,Pledge Creation,Pledge,Equity Shares,0.0,0


---
## Preparing Dataset For Further Analysis.

In [56]:
df = df[df['ACQUISITION_TRANSACTION_TYPE'] == 'Buy']
df = df[df['TYPE_OF_SECURITY'] == 'Equity Shares']
df = df[df['TYPE_OF_SECURITY'] == 'Equity Shares']
df = df[df['CATEGORY_OF_PERSON'].isin(['Promoters','Promoter Group'])]
df = df[df['MODE_OF_ACQUISITION'] == 'Market Purchase']


df = df[[
    'SYMBOL', 'NO_OF_SECURITIES_ACQUIRED', 'VALUE_OF_SECURITY_ACQUIRED',
    '%_SHAREHOLDING_PRIOR', '%_SHAREHOLDING_POST'
]]

In [57]:
a = df.copy()
df.head()

,SYMBOL,NO_OF_SECURITIES_ACQUIRED,VALUE_OF_SECURITY_ACQUIRED,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST
18,GODREJAGRO,3500.0,1779191,64.90,64.9
28,GNA,1100.0,736000,0.03,0.03
49,GODREJPROP,41600.0,49681632,47.33,47.34
55,GODREJAGRO,2798.0,1424445,64.90,64.9
56,DCW,573340.0,30001417,0.05,0.24


---
## Calculations Start Here.

### Finding Total Values And No Of Securities Bought.

In [58]:
df.head()

,SYMBOL,NO_OF_SECURITIES_ACQUIRED,VALUE_OF_SECURITY_ACQUIRED,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST
18,GODREJAGRO,3500.0,1779191,64.90,64.9
28,GNA,1100.0,736000,0.03,0.03
49,GODREJPROP,41600.0,49681632,47.33,47.34
55,GODREJAGRO,2798.0,1424445,64.90,64.9
56,DCW,573340.0,30001417,0.05,0.24


In [59]:
t_val = df[['SYMBOL','NO_OF_SECURITIES_ACQUIRED','VALUE_OF_SECURITY_ACQUIRED']]


In [60]:
t_val.head()
t_val = t_val.astype({'VALUE_OF_SECURITY_ACQUIRED':'float','NO_OF_SECURITIES_ACQUIRED':'float'})
t_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 18 to 4977
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   SYMBOL                      513 non-null    object 
 1   NO_OF_SECURITIES_ACQUIRED   513 non-null    float64
 2   VALUE_OF_SECURITY_ACQUIRED  513 non-null    float64
dtypes: float64(2), object(1)
memory usage: 16.0+ KB


In [61]:
val = pd.DataFrame(t_val.groupby('SYMBOL')['VALUE_OF_SECURITY_ACQUIRED'].sum()).reset_index()
val.head()

,SYMBOL,VALUE_OF_SECURITY_ACQUIRED
0,ADANIENT,1.278702e+10
1,ADANIGREEN,9.351393e+09
2,ADANIPORTS,8.293400e+04
3,ADANITRANS,3.565280e+05
4,AIROLAM,7.691300e+04


In [62]:
t_sec = pd.DataFrame(t_val.groupby('SYMBOL')['NO_OF_SECURITIES_ACQUIRED'].sum()).reset_index()
t_sec.head()

,SYMBOL,NO_OF_SECURITIES_ACQUIRED
0,ADANIENT,3486140.0
1,ADANIGREEN,4000100.0
2,ADANIPORTS,100.0
3,ADANITRANS,100.0
4,AIROLAM,1050.0


---

In [63]:
a_per_share_price = a[['SYMBOL','VALUE_OF_SECURITY_ACQUIRED','NO_OF_SECURITIES_ACQUIRED']]
a_per_share_price = a_per_share_price[a['VALUE_OF_SECURITY_ACQUIRED'] != '-']
a_per_share_price = a_per_share_price.astype({'VALUE_OF_SECURITY_ACQUIRED':'float','NO_OF_SECURITIES_ACQUIRED':'float'})

a_per_share_price

,SYMBOL,VALUE_OF_SECURITY_ACQUIRED,NO_OF_SECURITIES_ACQUIRED
18,GODREJAGRO,1779191.0,3500.0
28,GNA,736000.0,1100.0
49,GODREJPROP,49681632.0,41600.0
55,GODREJAGRO,1424445.0,2798.0
56,DCW,30001417.0,573340.0
...,...,...,...
4885,WALPAR,2001000.0,58000.0
4886,WALPAR,355000.0,10000.0
4905,MAHICKRA,105075.0,1500.0
4929,LIBAS,135000.0,5000.0


###  Average Price / Share

In [64]:
a_per_share_price['AVGPRICE_PERSHARE'] = a_per_share_price['VALUE_OF_SECURITY_ACQUIRED']/a_per_share_price['NO_OF_SECURITIES_ACQUIRED']

In [65]:
avg_per_share_price = a_per_share_price[['SYMBOL','AVGPRICE_PERSHARE']]
avg_per_share_price = avg_per_share_price.groupby(['SYMBOL']).mean()

In [66]:
# a_per_share_price.groupby('SYMBOL')['PERSHARE_PRICE'].unique()
avg_per_share_price.reset_index(inplace=True)

In [67]:
avg_per_share_price

,SYMBOL,AVGPRICE_PERSHARE
0,ADANIENT,3655.385437
1,ADANIGREEN,2346.793446
2,ADANIPORTS,829.340000
3,ADANITRANS,3565.280000
4,AIROLAM,73.250476
...,...,...
82,VIPIND,584.046503
83,VSTTILLERS,2515.020000
84,WALPAR,35.000000
85,WSTCSTPAPR,478.914900


###  Maximun And Minimun Price Per Share

In [68]:
highest_range  = pd.DataFrame(a_per_share_price.groupby('SYMBOL')['AVGPRICE_PERSHARE'].max()).reset_index()
highest_range['HIGH'] = round(highest_range['AVGPRICE_PERSHARE'],2)
highest_range = highest_range[['SYMBOL','HIGH']]
highest_range.head()

,SYMBOL,HIGH
0,ADANIENT,3851.91
1,ADANIGREEN,2424.44
2,ADANIPORTS,829.34
3,ADANITRANS,3565.28
4,AIROLAM,73.25


In [69]:
lowest_range = pd.DataFrame(a_per_share_price.groupby('SYMBOL')['AVGPRICE_PERSHARE'].min()).reset_index()
lowest_range['LOW'] = round(lowest_range['AVGPRICE_PERSHARE'],2)
lowest_range = lowest_range[['SYMBOL','LOW']]
lowest_range.head()

,SYMBOL,LOW
0,ADANIENT,3464.09
1,ADANIGREEN,2223.14
2,ADANIPORTS,829.34
3,ADANITRANS,3565.28
4,AIROLAM,73.25


In [70]:
highest_range.shape == lowest_range.shape == avg_per_share_price.shape

True

---
### Shareholding Percentage Prior And Post 

In [71]:
shareholding_prcnt = a[['SYMBOL','%_SHAREHOLDING_PRIOR','%_SHAREHOLDING_POST']]

In [72]:
shareholding_prcnt = shareholding_prcnt.astype({'%_SHAREHOLDING_PRIOR':'float','%_SHAREHOLDING_POST':'float'})

In [73]:
shareholding_prcnt['BUY%'] = shareholding_prcnt['%_SHAREHOLDING_POST'] - shareholding_prcnt['%_SHAREHOLDING_PRIOR']
shareholding_prcnt

,SYMBOL,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST,BUY%
18,GODREJAGRO,64.90,64.90,0.00
28,GNA,0.03,0.03,0.00
49,GODREJPROP,47.33,47.34,0.01
55,GODREJAGRO,64.90,64.90,0.00
56,DCW,0.05,0.24,0.19
...,...,...,...,...
4885,WALPAR,14.01,15.28,1.27
4886,WALPAR,15.28,15.50,0.22
4905,MAHICKRA,1.09,1.11,0.02
4929,LIBAS,0.00,0.03,0.03


In [74]:
prior  = pd.DataFrame(shareholding_prcnt.groupby('SYMBOL')['%_SHAREHOLDING_PRIOR'].max()).reset_index()
post  = pd.DataFrame(shareholding_prcnt.groupby('SYMBOL')['%_SHAREHOLDING_POST'].max()).reset_index()

In [75]:
prior

,SYMBOL,%_SHAREHOLDING_PRIOR
0,ADANIENT,0.32
1,ADANIGREEN,0.22
2,ADANIPORTS,0.00
3,ADANITRANS,0.00
4,AIROLAM,7.68
...,...,...
82,VIPIND,1.91
83,VSTTILLERS,22.02
84,WALPAR,15.28
85,WSTCSTPAPR,3.52


---
## Merging Sarts Here.

In [76]:
main_df = avg_per_share_price.merge(highest_range,on='SYMBOL')
main_df = main_df.merge(lowest_range,on='SYMBOL')
main_df = main_df.merge(prior,on='SYMBOL')
main_df = main_df.merge(post,on='SYMBOL')
main_df = main_df.merge(val,on='SYMBOL')
main_df = main_df.merge(t_sec,on='SYMBOL')





In [ ]:
main_df['%_BUY'] = main_df['%_SHAREHOLDING_POST'].astype(float) - main_df['%_SHAREHOLDING_PRIOR'].astype(float)

---
### Scrape Current Prices

In [77]:
y = list(main_df['SYMBOL'])

In [262]:
import time 

def realpriceupdate(y):
    price = {}
    not_happened = []
    for i in y:
        stockcode = str(i)
        stock_url  = f"https://www.screener.in/company/{stockcode}/"
        response = requests.get(stock_url)
        time.sleep(3)
        soup = BeautifulSoup(response.text, 'lxml')
        if len(soup.find_all('div',{"class":"flex flex-align-center"})) > 0:
            price[i] = (soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:])
        else:
            not_happened.append(i)

            
            
    return(price,not_happened)

In [264]:
got,not_got = realpriceupdate(y)

In [267]:
z = pd.DataFrame.from_dict([got])

In [268]:
z = z.T.reset_index()

In [269]:
z.columns = ['SYMBOL','CURRENT_PRICE']
z.head()

,SYMBOL,CURRENT_PRICE
0,ADANIENT,"3,278"
1,ADANIGREEN,"2,204"
2,ADANIPORTS,817
3,ADANITRANS,"3,279"
4,AIROLAM,101


In [270]:
z['CURRENT_PRICE'] = z['CURRENT_PRICE'].str.replace(',','')

In [272]:

z['CURRENT_PRICE'] = z['CURRENT_PRICE'].astype('float')

z.head()
main_df = main_df.merge(z,on='SYMBOL')


---

---

In [277]:
abv_10cr = main_df[main_df['VALUE_OF_SECURITY_ACQUIRED'] > 20000000]

In [279]:
less_thn_prmtr_price = abv_10cr[abv_10cr['AVGPRICE_PERSHARE'] > abv_10cr['CURRENT_PRICE']]

In [273]:
main_df.head()

,SYMBOL,AVGPRICE_PERSHARE,HIGH,LOW,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST,VALUE_OF_SECURITY_ACQUIRED,NO_OF_SECURITIES_ACQUIRED,CURRENT_PRICE
0,ADANIENT,3655.385437,3851.91,3464.09,0.32,0.35,1.278702e+10,3486140.0,3278.0
1,ADANIGREEN,2346.793446,2424.44,2223.14,0.22,0.25,9.351393e+09,4000100.0,2204.0
2,ADANIPORTS,829.340000,829.34,829.34,0.00,0.00,8.293400e+04,100.0,817.0
3,ADANITRANS,3565.280000,3565.28,3565.28,0.00,0.00,3.565280e+05,100.0,3279.0
4,AIROLAM,73.250476,73.25,73.25,7.68,0.01,7.691300e+04,1050.0,101.0


---

In [280]:
path = r'C:\Users\user\Desktop\Stocks_Automated\STOCKS_8OCT.csv'
path1 = r'C:\Users\user\Desktop\Stocks_Automated\LESS_PRMTR_AVGBUY_PRICE_8OCT.csv'
path2 = r'C:\Users\user\Desktop\Stocks_Automated\ABOVE2CR_8OCT.csv'

In [281]:
main_df.to_csv(path)
less_thn_prmtr_price.to_csv(path1)
abv_10cr.to_csv(path2)

# ENDS

---
### ROUGH BELOW...

In [282]:
# for i in z.CURRENT_PRICE:
#     if ',' in i:
#         z['new']= int(i.replace(',', ''))
#     else:
#         z['new']= float(i)
        
    
# z.head()

In [283]:
# for i in (z.CURRENT_PRICE):
#     print(i)
# z
    

In [284]:
# z['CURRENT_PRICE'] = int(z['CURRENT_PRICE'].replace(',', ''))
# # int(rectify.replace(',', ''))

In [285]:
# (price)

In [286]:
# prices = []
# for i in y:
# #     print(realpriceupdate(i))
#     prices.append(realpriceupdate(i))
    

In [287]:
# print(len(prices))
# len(y)
# print(realpriceupdate(y[23]))
# prices
# # res = dict(zip(y,prices))
# # res



# Pending...

In [288]:
# z['CURRENT_PRICE'] = z['CURRENT_PRICE'].str.strip(",")

In [289]:
# for i in z['CURRENT_PRICE']:
#     if ',' in i:
#         i.replace(',','')
#         print(i)

In [290]:
# abv_10cr['CURRENT_PRICE'].astype(float) < abv_10cr['AVGPRICE_PERSHARE']

In [291]:
# pending

# Pending...

In [292]:
# main_df = main_df.merge(z,on='SYMBOL')


In [293]:
# def realpriceupdate(i):
#     stockcode = str(i)
#     stock_url  = f"https://www.screener.in/company/{stockcode}/"
#     response = requests.get(stock_url)
#     soup = BeautifulSoup(response.text, 'lxml')
#     price = []
#     if len(soup.find_all('div',{"class":"flex flex-align-center"})) > 0:
        
# #         price = soup.find_all('div',{"class":"flex flex-align-center"})[0]
#         price = soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:]
#     else:
#         realpriceupdate(stockcode)
    


#     return(price)

In [294]:


# def realpriceupdate(y):
#     price = {}
#     not_happened = []
#     for i in y:
#         stockcode = str(i)
#         stock_url  = f"https://www.screener.in/company/{stockcode}/"
#         response = requests.get(stock_url)
#         soup = BeautifulSoup(response.text, 'lxml')
#         if len(soup.find_all('div',{"class":"flex flex-align-center"})) > 0:
#             price[i] = (soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:])
# #             price.append(soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:])
#         else:
#             not_happened.append(i)

            
            
#     return(price,not_happened)


In [295]:
# a,b = realpriceupdate(y)

In [296]:
# a1,b1 = realpriceupdate(b)

In [297]:
# price = {**a,**a1}


In [298]:
# price

In [299]:
# z.info()

In [300]:
# import time 

# def realpriceupdate(i):
#     stockcode = str(i)
#     stock_url  = f"https://www.screener.in/company/{stockcode}/"
#     response = requests.get(stock_url)
#     soup = BeautifulSoup(response.text, 'lxml')
#     price = soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:]
#     return(price)


In [301]:
# stockcode = "DHAMPURSUG"
# print(stockcode)

In [302]:
# # stock_url  = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol='+str(stockcode)
# stock_url  = f"https://www.screener.in/company/{stockcode}/"

# print(stock_url)

# response = requests.get(stock_url)


In [303]:
# print(response)

In [304]:
# soup = BeautifulSoup(response.text, 'lxml')

In [305]:
# price = soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:]

In [306]:
# print(price)

In [307]:
# def real_price():
#     stockcode = "DHAMPURSUG"
#     stock_url  = f"https://www.screener.in/company/{stockcode}/"
#     response = requests.get(stock_url)
#     soup = BeautifulSoup(response.text, 'lxml')
#     price = soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:]
#     print(price)

    